In [49]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service as ChromeService
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.by import By
import pandas as pd

import warnings
warnings.filterwarnings("ignore")

In [8]:
driver = webdriver.Chrome(service=ChromeService(ChromeDriverManager().install()))

driver.get("https://www.billboard.com/artist/nirvana/")

In [10]:
def extract_data(chart_name = ""):
    data = []
    
    table = driver.find_element(By.CLASS_NAME, 'artist-chart-history-items')
    rows = table.find_elements(By.XPATH, '*')
    
    for row in rows:
        track = {}
        track_name = row.find_element(By.ID, 'title-of-a-story')
        debut_date = row.find_element(By.CLASS_NAME, 'artist-chart-row-debut-date')
        peak_pos = row.find_element(By.CLASS_NAME, 'artist-chart-row-peak-pos')
        peak_date = row.find_element(By.CLASS_NAME, 'artist-chart-row-peak-date')
        weaks_on_chart = row.find_element(By.CLASS_NAME, 'artist-chart-row-week-on-chart')

        track['chart'] = chart_name
        track['track_name'] = track_name.text
        track['debut_date']  = debut_date.text
        track['peak_pos'] = peak_pos.text
        track['peak_date'] = peak_date.text
        track['weaks_on_chart'] = weaks_on_chart.text
        data.append(track)

    
    return data
        
        # print(row)
        
    print()

In [22]:

filter = driver.find_element(By.XPATH, '//*[@id="artist-chart-selector"]')
current_li = 1

all_data = []

while(True):
    if current_li == 55:
        break
        
    filter.click()
    
    li_path = f'//*[@id="main-wrapper"]/main/div[2]/div[2]/div/div[2]/div[1]/nav/ul/li[{current_li}]/a'
    new_li = driver.find_element(By.XPATH, li_path)
    new_li.click()

    print(f"Chart: {new_li.get_attribute('text').strip()}", end='\r')
    
    all_data.extend(extract_data(chart_name=new_li.get_attribute('text').strip()))
    
    current_li += 1
driver.refresh()

In [23]:
import pandas as pd

df = pd.DataFrame(all_data)

In [24]:
df

,chart,track_name,debut_date,peak_pos,peak_date,weaks_on_chart
0,Billboard Hot 100,Smells Like Teen Spirit,12.07.91,6,01.11.92,20
1,Billboard Hot 100,You Know You're Right,10.19.02,45,11.23.02,20
2,Billboard Hot 100,Come As You Are,03.21.92,32,05.02.92,18
3,Billboard Hot 100,Lithium,08.08.92,64,08.15.92,9
4,Billboard Hot 100,Something In The Way,03.26.22,46,03.26.22,1
...,...,...,...,...,...,...
135,Billboard Global 200 Artists,Nirvana,03.19.22,7,03.26.22,3
136,Rock & Alternative Producers,Nirvana,03.19.22,7,03.26.22,3
137,The Official U.K. Albums Chart,Nirvana,03.19.22,7,03.26.22,3
138,Germany Albums,Nirvana,03.19.22,7,03.26.22,3


In [25]:
df['chart'].value_counts()
useful_charts = [
    'Mainstream Rock Airplay',
    'Alternative Airplay',
    'Radio Songs', 
    'LyricFind U.S.',
    'Billboard Hot 100',
    'Rock Digital Song Sales',
    'Hot Alternative Songs',
    'LyricFind Global',
    'Alternative Digital Song Sales',
    'Billboard Global 200',
    'Canadian Digital Song Sales',
    'Hot Rock Songs',
    'Hot Rock & Alternative Songs',
    'Digital Song Sales',
    'Rock Streaming Songs',
    'Alternative Streaming Songs'
    ]
indices = df.loc[df['chart'].isin(useful_charts),].index

In [26]:
df = df.loc[indices,]

In [27]:
df.to_csv('billboard.csv',index=False)

### Combining Billboard data with all data

In [102]:
nirvana = pd.read_csv('nirvana_v4.csv')
nirvana.drop_duplicates(subset=['track_name', 'album_name', 'release_year'], inplace=True)

In [103]:
df.reset_index(drop=True, inplace=True)


In [104]:
for i in range(df.shape[0]):

    if nirvana.loc[nirvana['track_name'].str.title() == df.loc[i,'track_name'],].index.size == 1:
        # If song has only one occurence in nirvana_v4 then input Billboard data there
        chart = df.loc[i,'chart'].lower().replace(' ','_')
        track = df.loc[i,'track_name']
        index = nirvana.loc[nirvana['track_name'].str.title() == df.loc[i,'track_name'],].index[0]

        if chart + '_debut_date' not in nirvana.columns: nirvana[chart + '_debut_date'] = ''
        nirvana[chart + '_debut_date'][index] = df.loc[i,'debut_date']

        if chart + '_peak_pos' not in nirvana.columns: nirvana[chart + '_peak_pos'] = ''
        nirvana[chart + '_peak_pos'][index] = df.loc[i,'peak_pos']

        if chart + '_peak_date' not in nirvana.columns: nirvana[chart + '_peak_date'] = ''
        nirvana[chart + '_peak_date'][index] = df.loc[i,'peak_date']

        if chart + '_weaks_on_chart' not in nirvana.columns: nirvana[chart + '_weaks_on_chart'] = ''
        nirvana[chart + '_weaks_on_chart'][index] = df.loc[i,'weaks_on_chart']
    else:
        # If song has multiple occurences in nirvana_v4 find the original (lowest release year)
        chart = df.loc[i,'chart'].lower().replace(' ','_')
        track = df.loc[i,'track_name']
        original_release_year = min(nirvana.loc[nirvana.loc[nirvana['track_name'].str.lower() == track.lower(),].index, 'release_year'])
        # print(nirvana.loc[nirvana.loc[nirvana['track_name'].str.title() == track.title(),].index, 'release_year'], track)
        # print()
        index = nirvana.loc[(nirvana['release_year'] == original_release_year) & (nirvana['track_name'].str.lower() == track.lower()),].iloc[:,:5].index

        if chart + '_debut_date' not in nirvana.columns: nirvana[chart + '_debut_date'] = ''
        nirvana[chart + '_debut_date'][index] = df.loc[i,'debut_date']

        if chart + '_peak_pos' not in nirvana.columns: nirvana[chart + '_peak_pos'] = ''
        nirvana[chart + '_peak_pos'][index] = df.loc[i,'peak_pos']

        if chart + '_peak_date' not in nirvana.columns: nirvana[chart + '_peak_date'] = ''
        nirvana[chart + '_peak_date'][index] = df.loc[i,'peak_date']

        if chart + '_weaks_on_chart' not in nirvana.columns: nirvana[chart + '_weaks_on_chart'] = ''
        nirvana[chart + '_weaks_on_chart'][index] = df.loc[i,'weaks_on_chart']

In [105]:
# indices = nirvana.loc[nirvana['album_name'].str.contains('Remastered')].index
# nirvana.drop(index=indices, inplace=True)

In [106]:
nirvana.to_csv('nirvana_v5.csv', index=False)